In [43]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
import pandas as pd
from hkfold import HKFold
from sklearn.model_selection import KFold
from keras.layers import PReLU
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ReduceLROnPlateau

In [19]:
dframe_path = 'data/cabauw/processed.csv'

try:
    df = pd.read_csv(dframe_path, na_values='--')
except UnicodeDecodeError:
    df = pd.read_csv(dframe_path, na_values='--', compression='gzip')


df = df[(df.ustar > 0.1) & (abs(df.H) > 10) & (df.wind > 1)]
df = df[df.ds != 201603]
#df = df.sort_values(['ds', 'tt'])
df = df.sample(frac=1)  # aka shuffle

In [55]:
features = [
    'air_dens', 'surf_temp', 'spec_hum', 'rel_hum', 'press', 'rain', 'soil_temp',
    'wind',
    #'ustar', 'H', 'LE', 'zL_m', 'lval'
]

target = 'phi_m'

train_idx, test_idx = next(HKFold(KFold(50, shuffle=True), h=int(1 * 60 / 10)).split(df))

nr, ne, nn = len(train_idx), len(test_idx), len(df)
print('training with %d (%.1f%%) samples, testing with %d (%.1f%%), %d (%.1f%%) samples lost' % (
    nr, 100 * nr / nn, ne, 100 * ne / nn, nn - nr - ne, 100 * (nn - nr - ne) / nn
))

train_x, train_y = df[features].iloc[train_idx], df[target].iloc[train_idx]
test_x, test_y = df[features].iloc[test_idx], df[target].iloc[test_idx]

mean_x, mean_y = train_x.mean(), train_y.mean()
std_x, std_y = train_x.std(), train_y.std()

train_x = (train_x - mean_x) /  std_x
test_x = (test_x - mean_x) / std_x

#train_y = (train_y - mean_y) / std_y
#test_y = (test_y - mean_y) / std_y

training with 384783 (76.9%) samples, testing with 10009 (2.0%), 105644 (21.1%) samples lost


In [68]:
model = Sequential([
    Dense(1024, input_shape=(len(features),)), PReLU(),
    Dense(1024), PReLU(),
    Dense(1024), PReLU(),
    Dense(1024), PReLU(),
    Dense(1024), PReLU(),
    Dense(1024), PReLU(),
    Dense(1024), PReLU(),
    Dense(1)
])

opt = RMSprop(lr=0.001)
model.compile(loss='mse', optimizer=opt)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_153 (Dense)            (None, 1024)              9216      
_________________________________________________________________
p_re_lu_113 (PReLU)          (None, 1024)              1024      
_________________________________________________________________
dense_154 (Dense)            (None, 1024)              1049600   
_________________________________________________________________
p_re_lu_114 (PReLU)          (None, 1024)              1024      
_________________________________________________________________
dense_155 (Dense)            (None, 1024)              1049600   
_________________________________________________________________
p_re_lu_115 (PReLU)          (None, 1024)              1024      
_________________________________________________________________
dense_156 (Dense)            (None, 1024)              1049600   
__________

In [69]:
model.fit(
    train_x, train_y,
    batch_size=256, epochs=250,
    callbacks=[ReduceLROnPlateau(factor=0.5, verbose=1, min_lr=1e-6)],
    validation_data=(test_x, test_y)
)

Train on 384783 samples, validate on 10009 samples
Epoch 1/250
384783/384783 [==============================] - 44s 113us/step - loss: 1.8444 - val_loss: 0.7016
Epoch 2/250
384783/384783 [==============================] - 42s 110us/step - loss: 0.6627 - val_loss: 0.6766
Epoch 3/250
384783/384783 [==============================] - 42s 110us/step - loss: 0.6624 - val_loss: 0.7714
Epoch 4/250
384783/384783 [==============================] - 43s 113us/step - loss: 0.6501 - val_loss: 0.6609
Epoch 5/250
384783/384783 [==============================] - 43s 111us/step - loss: 0.6586 - val_loss: 0.6489
Epoch 6/250
384783/384783 [==============================] - 42s 109us/step - loss: 0.6610 - val_loss: 0.6509
Epoch 7/250
384783/384783 [==============================] - 42s 109us/step - loss: 1.6449 - val_loss: 0.8712
Epoch 8/250
384783/384783 [==============================] - 42s 110us/step - loss: 0.6531 - val_loss: 0.9489
Epoch 9/250
384783/384783 [==============================] - 42s 110u

KeyboardInterrupt: 